In [144]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import MultinomialNB

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.gaussian_process import GaussianProcessClassifier
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score, confusion_matrix,classification_report

import nltk
import nltk as nlp
import string
import re

In [163]:
train = pd.read_csv('resources/data/train.csv')

rt = r'rt'
train['message'] = train['message'].str.lower()
train['message'] = train['message'].replace(to_replace = rt, value = '', regex = True)
#train['message'] = train['message'].str.replace(r'@', 'twithandle ')

# Remove stop words
def remove_stop_words(word):
    if word not in stop:
        return word
    else:
        return ''
    
sid = SentimentIntensityAnalyzer()
def sentiment_score(sentence):
    
    return sid.polarity_scores(sentence)['compound']

def avg_word_len(sentence):
    """takes in a sentence, tokenizes it and computes
        the average word length of the sentence
        
        parameters:
        -----------
        sentence: str
    
        returns:
        --------
        avg word length: float        
    """  
    word_list = word_tokenize(sentence)
    length = 0
    for word in word_list:
        length += len(word)
        
    return length/len(word_list)
    
stop = stopwords.words('english')

train['message'] = [' '.join([remove_stop_words(word) for word in text.split(' ')])for text in train['message']]
train['sentiment_compound'] = train['message'].apply( sentiment_score)
train['avg_word_len'] = train['message'].apply(avg_word_len)
train['length'] = train['message'].apply(len)

train.head()

,sentiment,message,tweetid,sentiment_compound,avg_word_len,length
0,-1,@darreljorstad: funny hell! canada demands '...,897853122080407553,-0.4574,5.000000,120
1,-1,biggest lies climate change global warming...,925046776553529344,-0.2960,7.000000,106
2,-1,coming revelation $q$global warming$q$ #fra...,696354236850786305,0.0000,4.071429,131
3,-1,@dineshdsouza: let's see world ends @reald...,846806509732483072,0.0000,4.120000,123
4,-1,@stevesgoddard: obama control climate. ...,628085266293653504,-0.6249,5.437500,105


In [164]:
x_train,x_test,y_train,y_test = train_test_split(train.message, train.sentiment, test_size=0.2, random_state=2020)

pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', LogisticRegression())])

model = pipe.fit(x_train, y_train)
prediction = model.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))

accuracy: 71.2%


/home/mukovhe/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [173]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', LinearSVC(loss = 'squared_hinge',C = 1.1))])

model = pipe.fit(x_train, y_train)
prediction = model.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))


#save model
import pickle

model_save_path = "resources/models/linearSVC.pkl"
with open(model_save_path,'wb') as file:
    pickle.dump(model,file)

accuracy: 72.87%


In [166]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', MultinomialNB())])

model = pipe.fit(x_train, y_train)
prediction = model.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))

accuracy: 58.52%


In [167]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', BernoulliNB())])

model = pipe.fit(x_train, y_train)
prediction = model.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))

accuracy: 62.81%


In [168]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', GradientBoostingClassifier(loss = 'deviance',
                                                   learning_rate = 0.01,
                                                   n_estimators = 10,
                                                   max_depth = 5,
                                                   random_state=55))])

model = pipe.fit(x_train, y_train)
prediction = model.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))

accuracy: 52.5%


In [169]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', XGBClassifier(loss = 'deviance',
                                                   learning_rate = 0.01,
                                                   n_estimators = 10,
                                                   max_depth = 5,
                                                   random_state=2020))])

model = pipe.fit(x_train, y_train)
prediction = model.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))

[01:56:59] WARNING: /workspace/src/learner.cc:480: 
Parameters: { loss } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


accuracy: 57.35%


In [170]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', SGDClassifier())])

model = pipe.fit(x_train, y_train)
prediction = model.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))

accuracy: 70.74%


In [171]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', DecisionTreeClassifier(criterion= 'entropy',
                                           max_depth = 10, 
                                           splitter='best', 
                                           random_state=2020))])

model = pipe.fit(x_train, y_train)
prediction = model.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))

accuracy: 56.84%


In [172]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', RandomForestClassifier(criterion='entropy',n_estimators = 150))])

model = pipe.fit(x_train, y_train)
prediction = model.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))

accuracy: 68.27%


In [46]:
X = train.message,
Y = train.sentiment
#le = LabelEncoder()
#Y = le.fit_transform(Y)
#Y = Y.reshape(-1,1)

In [139]:
x_train,x_test,y_train,y_test = train_test_split(train.message, train.sentiment, test_size=0.15, random_state=2020)
max_words = 140
max_len = 75
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(x_train)
sequences = tok.texts_to_sequences(x_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(512)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model
model = RNN()


In [140]:
from tensorflow.keras.utils import plot_model 
plot_model(model, to_file='model1.png')
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [141]:
model.fit(sequences_matrix,y_train,batch_size=256,epochs=2,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/2
82/82 [==============================] - 113s 1s/step - loss: 0.4277 - accuracy: 0.4968 - val_loss: -1.8508 - val_accuracy: 0.5141
Epoch 2/2
82/82 [==============================] - 111s 1s/step - loss: -37.2880 - accuracy: 0.4659 - val_loss: -106.3122 - val_accuracy: 0.4242


In [142]:
test_sequences = tok.texts_to_sequences(x_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
accr = model.evaluate(test_sequences_matrix,y_test)
print('Accuracy: {:0.2f}'.format(accr[1]))


145/145 [==============================] - 20s 141ms/step - loss: -116.3992 - accuracy: 0.4244
Accuracy: 0.42
